In [1]:
# 6-1
!mkdir chap6
%cd ./chap6

/Users/maton/BERT_Practice/chap6


/Users/maton/.pyenv/versions/3.10.13/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
# 6-3
import random
import glob
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer, BertForSequenceClassification
import pytorch_lightning as pl

# 日本語の事前学習モデル
MODEL_NAME = "tohoku-nlp/bert-base-japanese-whole-word-masking"

In [4]:
# 6-4（説明コメント付き：日本語BERTによる二値分類のセットアップ）
# =====================================================================
# 目的：
#  - 事前学習済みの日本語BERT（MODEL_NAME）を、Sequence Classification（文分類）タスク用ヘッド付きでロードし、
#    デバイス（MacのMPS / CUDA / CPU）に配置する。
#
# 理論メモ：
#  - BertForSequenceClassification は BERT 本体の上に「分類ヘッド（Dropout→Linear）」を載せたモデルである。
#  - 推論時の出力 logits は形状 [B, num_labels]。num_labels=2 のとき、
#    softmax(logits) によりクラス確率（2クラス）が得られる（argmax で予測ラベル）。
#  - 学習時の損失は入力ラベルと num_labels により自動選択される（HFの既定）：
#      * num_labels == 1 かつラベルが float       → 回帰（MSELoss）
#      * num_labels > 1 かつラベルが整数カテゴリ → 単一ラベル分類（CrossEntropyLoss）
#      * それ以外（ラベルが multi-hot 等）      → マルチラベル（BCEWithLogitsLoss）
#  - BERT の文表現は概ね [CLS] トークンの隠れ状態（最終層）を用い、分類ヘッドで写像する設計（プール層の出力を使う実装もある）。
#  - トークナイザは [CLS]/[SEP]/[PAD] を自動付与（既定）し、attention_mask でPADを無視する。
# =====================================================================

import torch
from transformers import BertJapaneseTokenizer, BertForSequenceClassification


# --- デバイス選択（Mac優先：MPS → CUDA → CPU） ---
def get_best_device() -> torch.device:
    # Apple Silicon + macOS Metal（CUDAは不可）
    if torch.backends.mps.is_available() and torch.backends.mps.is_built():
        return torch.device("mps")
    # 他環境（Linux/Windows等）ではCUDAを優先
    if torch.cuda.is_available():
        return torch.device("cuda")
    # それ以外はCPU
    return torch.device("cpu")


device = get_best_device()
print(f"[info] using device = {device}")

# --- モデル名の決定（未定義時の保険） ---
try:
    MODEL_NAME
except NameError:
    # WWM版：語単位での一貫性を意識したマスキングで事前学習（日本語に適した設定）
    MODEL_NAME = "tohoku-nlp/bert-base-japanese-whole-word-masking"

# --- トークナイザのロード ---
# 形態素解析→WordPieceの二段分割。語彙はモデルとペアで一致させる（IDずれ防止）。
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)

# --- 文分類モデル（2クラス）をロード ---
# num_labels=2：二値分類（例：ネガ/ポジ、真/偽 など）
# 出力 logits 形状：[B, 2]。学習時にラベルが整数（0/1）なら CrossEntropyLoss が自動選択される。
bert_sc = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

# --- モデルを最適デバイスへ配置 ---
# 元コード：bert_sc = bert_sc.cuda()
# → Mac の場合 .cuda() は不可。統一して .to(device) を用いる。
bert_sc = bert_sc.to(device)

# --- 推論時の基本姿勢 ---
# 評価モードにして Dropout を停止（推論の安定化）。学習時は bert_sc.train() に切り替える。
bert_sc.eval()

# ---------------------------------------------------------------------
# 参考：推論の最小例（コメント解除で動作確認）
# ---------------------------------------------------------------------
# texts = ["今日は自然言語処理を学ぶ。", "この映画は退屈だった。"]
# enc = tokenizer(
#     texts,
#     max_length=128,           # 学習・評価で固定長を揃えると効率的（pad_to_max_length 相当）
#     padding="max_length",
#     truncation=True,
#     return_tensors="pt"
# )
# enc = {k: v.to(device) for k, v in enc.items()}
# with torch.no_grad():
#     logits = bert_sc(**enc).logits           # [B, 2]
#     probs  = logits.softmax(dim=-1)          # クラス確率
#     preds  = probs.argmax(dim=-1)            # 予測クラスID（0/1）
# print("probs:", probs.detach().cpu().numpy())
# print("preds:", preds.detach().cpu().tolist())

# ---------------------------------------------------------------------
# 運用上の注意（理論と実務の橋渡し）
# ---------------------------------------------------------------------
# - 前処理の一貫性：学習・評価で tokenizer の設定（max_length, padding, truncation）が一致していること。
# - ラベル整合性：データセット内で 0/1 とクラス名のマッピングを厳密に固定（モデル保存時に一緒に管理）。
# - 不均衡データ：クラス不均衡が強い場合は、重み付き損失（class_weight）や閾値最適化で改善。
# - 再現性：random seed、モデルバージョン、辞書バージョンを固定。デバイス差（MPS/CUDA/CPU）で微差が出る点は許容範囲で管理。
# - 転移学習：タスクに合わせて最終ヘッドのみならずBERT側も微調整（学習率は本体を小さく、ヘッドを大きくするのが通例）。

[info] using device = mps


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at tohoku-nlp/bert-base-japanese-whole-word-masking and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [5]:
# 6-5（説明コメント付き：二値文分類のバッチ推論と精度計算）
# =====================================================================
# 目的：
#  - 日本語BERTの文分類モデル（bert_sc; num_labels=2）を用いて、複数文の推論を一括で行い、
#    予測ラベルと単純精度（accuracy）を計算・表示する。
#
# 理論メモ：
#  - BertForSequenceClassification の出力 logits 形状は [B, num_labels]。
#    num_labels=2 のとき argmax(logits) は「2クラスsoftmaxの最大事後確率クラス」と一致する。
#  - tokenizer(..., padding='longest') はバッチ内の最長系列に合わせてPADを付与する
#    “動的パディング”。attention_mask により PAD 位置は Self-Attention から無視される。
#  - 学習時はラベル（0/1）が整数クラスなら CrossEntropyLoss が自動選択される（HFの既定）。
#    本セルは推論のみ（no_grad & eval）。
#  - デバイス（MPS/CUDA/CPU）は“モデルが現在置かれている device”に合わせて入力テンソルを移動する。
#    （Mac は通常 MPS。`.cuda()` は不可なので `.to(device)` を一貫して用いる。）
# =====================================================================

text_list = [
    "この映画は面白かった。",  # 正例のつもり（1）
    "この映画の最後にはがっかりさせられた。",  # 負例のつもり（0）
    "この映画を見て幸せな気持ちになった。",  # 正例のつもり（1）
]
label_list = [1, 0, 1]  # 教師ラベル（0=negative, 1=positive の想定）

# --- データの符号化（トークン化→ID化） ---
# padding='longest'：バッチ内の最長文の長さに合わせてPADを付与（動的パディング）。
# return_tensors='pt'：PyTorchテンソルで返す（input_ids/attention_mask/token_type_ids など）。
# ＊長文や学習時は max_length と truncation=True を併用して長さ上限を明示するのが安定。
encoding = tokenizer(text_list, padding="longest", return_tensors="pt")

# --- デバイスをモデルに揃える（MPS/CUDA/CPU いずれでも動く書き方） ---
# bert_sc は 6-4 で .to(device) 済みの前提。ここではその device を参照して入力を移す。
device = next(bert_sc.parameters()).device
encoding = {k: v.to(device) for k, v in encoding.items()}

# ラベルテンソル作成
# dtype=torch.long：分類（CE Loss）のクラスIDは long 整数で表現するのが通例。
labels = torch.tensor(label_list, dtype=torch.long, device=device)

# --- 推論（評価モード＋勾配無効） ---
# eval()：Dropout等を停止して推論安定化。6-4で eval 済みでも冪等。
bert_sc.eval()
with torch.no_grad():
    # モデル呼び出し：.forward(**encoding) と bert_sc(**encoding) は等価（後者が慣用的）
    output = bert_sc(**encoding)

# --- 出力の解釈と評価 ---
scores = output.logits  # 形状 [B, 2]：各クラスのスコア（ロジット）
labels_predicted = scores.argmax(-1)  # 予測クラスID（最尤クラス）
num_correct = (labels_predicted == labels).sum().item()  # 正解数（スカラー）
accuracy = num_correct / labels.size(0)  # 単純精度（micro accuracy）

# --- 可視出力（形状と予測・精度） ---
print("# scores (shape):")
print(scores.size())  # 例：torch.Size([3, 2])
print("# predicted labels:")
print(labels_predicted)  # 例：tensor([1, 0, 1], device=..., dtype=torch.int64)
print("# accuracy:")
print(accuracy)  # 例：1.0

# 運用上の注意：
# - データ分布が偏っている場合、単純精度だけでは不十分。適宜、適合率/再現率/F1、混同行列を併用する。
# - 推論の閾値調整が必要な場合（特にコスト非対称なタスク）は、2クラスsoftmaxの確率を取得して閾値最適化を行う。
# - 学習と評価で tokenizer の前処理（max_length/padding/truncation）を一致させること。
# - デバイス差（MPS/CUDA/CPU）でごく僅かな数値差が出る場合があるが、多くは統計誤差の範囲。

# scores (shape):
torch.Size([3, 2])
# predicted labels:
tensor([1, 1, 1], device='mps:0')
# accuracy:
0.6666666666666666


In [6]:
# 6-6（説明コメント付き：ラベル付き入力で損失を計算）
# =====================================================================
# 目的：
#  - 3本のテキストを一括でトークン化し、ラベル（0/1）を入力に同梱して
#    BertForSequenceClassification が内部で計算する損失（loss）を取得して表示する。
#
# 理論メモ：
#  - Hugging Face の BertForSequenceClassification は、forward(**batch) に
#    'labels' を渡すと自動で損失を計算して返す。
#    * num_labels=2 かつ labels が整数クラス（dtype: long）の場合 → CrossEntropyLoss
#    * 出力 logits の形状は [B, num_labels]。loss はバッチ平均のスカラー。
#  - tokenizer(..., padding='longest') は“動的パディング”（バッチ内の最長系列に合わせてPAD）。
#    attention_mask により PAD 位置の注意は抑制される。
#  - デバイス整合：モデル（bert_sc）の置かれている device（MPS/CUDA/CPU）に
#    入力テンソルを合わせること（.to(device)）。Mac では .cuda() は不可。
# =====================================================================

# 符号化（トークン化→ID化）。長さはバッチ内最長に合わせてPAD（学習では max_length+truncation推奨）。
encoding = tokenizer(text_list, padding="longest", return_tensors="pt")

# ラベルを入力 dict に追加。
# - CrossEntropyLoss の前提として dtype は long（整数クラスID）。
# - デバイス移動はこの後まとめて行うので、ここではCPUでOK。
encoding["labels"] = torch.tensor(label_list, dtype=torch.long)

# デバイス整合：モデルの実デバイスへ（Macは多くの場合 'mps'）。
device = next(bert_sc.parameters()).device
encoding = {k: v.to(device) for k, v in encoding.items()}

# ロスの計算
# - labels を渡しているため、出力に loss が含まれる。
# - 学習（逆伝播）するなら bert_sc.train() にして、loss.backward() → optimizer.step() を行う。
#   ここでは損失値の確認のみ。
output = bert_sc(**encoding)
loss = output.loss  # スカラー損失（バッチ平均）

print(loss)

# 補足（運用上の注意）：
# - 動的パディングはバッチごとに系列長が変わるため、学習ループでは DataCollator で揃えるか、
#   max_length + truncation + padding='max_length' で固定長にすると効率が安定する。
# - クラス不均衡が大きい場合、重み付き損失や閾値最適化、評価指標（F1等）の併用が有効。

tensor(0.5852, device='mps:0', grad_fn=<NllLossBackward0>)
